In [2]:
import cv2 as cv
import numpy as np
import imgaug as iaa
import matplotlib.pyplot as plt
from osgeo import gdal,ogr,osr

In [3]:
id_mosaico = '212750'
file_shp_1 = '../../sensix_daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_1.shp'
file_shp_2 = '../../sensix_daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_2/daninha_2.shp'
name_mosaico = '../../sensix_daninhas/ortomosaicos/pre-emergente/'+ id_mosaico + '/ortomosaico.tif'
output_dataset = '../../sensix_daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/dataset'
tif_generated_1 = '../../sensix_daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_1.tif'
tif_generated_2 = '../../sensix_daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_2.tif'

MAX_PIXELS_DANINHA = 5000

In [4]:
def GetExtent(gt,cols,rows):
    ''' Return list of corner coordinates from a geotransform

        @type gt:   C{tuple/list}
        @param gt: geotransform
        @type cols:   C{int}
        @param cols: number of columns in the dataset
        @type rows:   C{int}
        @param rows: number of rows in the dataset
        @rtype:    C{[float,...,float]}
        @return:   coordinates of each corner
    '''
    ext=[]
    xarr=[0,cols]
    yarr=[0,rows]

    for px in xarr:
        for py in yarr:
            x=gt[0]+(px*gt[1])+(py*gt[2])
            y=gt[3]+(px*gt[4])+(py*gt[5])
            ext.append([x,y])
            print(x,y)
        yarr.reverse()
    return ext

In [5]:
def ReprojectCoords(coords,src_srs,tgt_srs):
    ''' Reproject a list of x,y coordinates.

        @type geom:     C{tuple/list}
        @param geom:    List of [[x,y],...[x,y]] coordinates
        @type src_srs:  C{osr.SpatialReference}
        @param src_srs: OSR SpatialReference object
        @type tgt_srs:  C{osr.SpatialReference}
        @param tgt_srs: OSR SpatialReference object
        @rtype:         C{tuple/list}
        @return:        List of transformed [[x,y],...[x,y]] coordinates
    '''
    trans_coords=[]
    transform = osr.CoordinateTransformation( src_srs, tgt_srs)
    for x,y in coords:
        x,y,z = transform.TransformPoint(x,y)
        trans_coords.append([x,y])
    return trans_coords

In [6]:
ds=gdal.Open(name_mosaico)
gt=ds.GetGeoTransform()
cols = ds.RasterXSize
rows = ds.RasterYSize
ext=GetExtent(gt,cols,rows)

src_srs=osr.SpatialReference()
src_srs.ImportFromWkt(ds.GetProjection())
tgt_srs = src_srs.CloneGeogCS()

geo_ext= ReprojectCoords(ext,src_srs,tgt_srs)

xmin = geo_ext[0][0]
ymax = geo_ext[0][1]
xmax = geo_ext[2][0]
ymin = geo_ext[1][1]

pixel_size_x = gt[1]
pixel_size_y = gt[5]

-52.70069034777114 -28.668701391937528
-52.70069034777114 -28.67501699726216
-52.69167850997385 -28.67501699726216
-52.69167850997385 -28.668701391937528


In [7]:
!gdal_rasterize -burn 255 -ot Byte -tr $pixel_size_x $pixel_size_y -te $xmin $ymin $xmax $ymax $file_shp_1 $tif_generated_1

#!gdal_rasterize -burn 255 -ot Byte -tr $pixel_size_x $pixel_size_y -te $xmin $ymin $xmax $ymax $file_shp_2 $tif_generated_2

0...10...20...30...40...50...60...70...80...90...100 - done.


In [8]:
daninha_1 = gdal.Open(tif_generated_1)
daninha_band_1 = daninha_1.GetRasterBand(1)

#daninha_2 = gdal.Open(tif_generated_2)
#daninha_band_2 = daninha_2.GetRasterBand(1)

mosaico = gdal.Open(name_mosaico)

red = mosaico.GetRasterBand(1)
green = mosaico.GetRasterBand(2)
blue = mosaico.GetRasterBand(3)
alpha = mosaico.GetRasterBand(4)


In [9]:
count = 0
for x in range(0,mosaico.RasterXSize,100):
    for y in range(0,mosaico.RasterYSize,100):
        if count > 10000:
            break
            
        if ((x+100)>mosaico.RasterXSize) or ((y+100)>mosaico.RasterYSize):
            continue
            
        daninha_parcela = daninha_band_1.ReadAsArray(x,y,100,100) #+ daninha_band_2.ReadAsArray(x,y,100,100)
        blueparcela = blue.ReadAsArray(x,y,100,100)
        greenparcela = green.ReadAsArray(x,y,100,100)
        redparcela = red.ReadAsArray(x,y,100,100)
        imgparcela = cv.merge((blueparcela,greenparcela,redparcela))
        
        if (cv.countNonZero(blueparcela) + cv.countNonZero(greenparcela) + cv.countNonZero(redparcela)) < 24000:
            continue
            
        if cv.countNonZero(daninha_parcela) > MAX_PIXELS_DANINHA:
            cv.imwrite(output_dataset + '/sim/daninha_{x}_{y}.png'.format(x=x,y=y),imgparcela)
            count += 1
            
        else:
            cv.imwrite(output_dataset + '/nao/daninha_{x}_{y}.png'.format(x=x,y=y),imgparcela)
            count += 1

KeyboardInterrupt: 